# 準備

In [ ]:
# 必要なライブラリのインストール
! pip install pixivpy --upgrade

In [ ]:
# データベースのダウンロード
! wget https://github.com/kosuke1701/illust-search/releases/download/0.0/vectors.sql

In [ ]:
from getpass import getpass
import io
import sqlite3

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from pixivpy3 import PixivAPI

In [ ]:
# Following codes which define `array` type in sqlite3 is copied from the following Stack Overflow:
# https://stackoverflow.com/questions/18621513
# question by:
# Joe Flip (https://stackoverflow.com/users/1715453/joe-flip)
# answered by:
# unutbu (https://stackoverflow.com/users/190597/unutbu)
def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())
def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)
# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)
# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

In [ ]:
# Load database
dim = 500
data_dtype = [("id", int), ("face", int), ("xmin", int), ("xmax", int), \
              ("ymin", int), ("ymax", int), ("vector", np.float32, dim)]

with sqlite3.connect("vectors.sql", detect_types=sqlite3.PARSE_DECLTYPES) as conn:
    c = conn.cursor()

    c.execute("SELECT * FROM face")
    _data = c.fetchall()
    data = np.array(_data, dtype=data_dtype)


In [ ]:
# イラストが既に削除されている場合はAssertionErrorが返される
def download(id, username, password):
    aapi = PixivAPI()
    aapi.login(username, password)
    illust = aapi.works(id)
    assert illust["status"] == "success", "AppPixivAPI.works({}) returns failure status.".format(id)
    illust = illust.response[0]
    #print(illust)
    url = illust["image_urls"]["large"]
    file_type = url.split(".")[-1]

    # Download illust.
    fn = f"{id}.{file_type}"
    aapi.download(url, path=".", name=fn)

    return fn

In [ ]:
# Test
username = getpass("Username:")
password = getpass("Password:")
i_illust = 67108889
xmin, xmax,ymin, ymax = 366, 695, 61, 399
fn = download(i_illust, username, password)
im = Image.open(fn)
im = im.crop((xmin, ymin, xmax, ymax))
im = im.resize((128,128))

display(im)

target_id = 30
i_illust = data["id"][target_id]
xmin, xmax,ymin, ymax = data["xmin"][target_id], data["xmax"][target_id], \
    data["ymin"][target_id], data["ymax"][target_id]
fn = download(i_illust, username, password)
im = Image.open(fn)
im = im.crop((xmin, ymin, xmax, ymax))
im = im.resize((128,128))

display(im)